# ELT Project

## Finding Data:

DATA SOURCE: https://www.kaggle.com/datasnaek/youtube-new/data <br/>
Utilizing: <br/>
3 csv files with Video Information (Canada, US, and Britain) <br/>
3 json files with Category Assignment (Canada, US, and Britain) <br/>

## Data Cleanup & Analysis

Plan and document the following:
* The sources of data that you will extract from.
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
* The type of final production database to load the data into (relational or non-relational).
* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report:

Submit a Final Report that describes the following:
* Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* Transform: what data cleaning or transformation was required.
* Load: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

In [1]:
import os
import pandas as pd
import json
import requests

from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [2]:
json_CA = os.path.join("data", "CA_category_id.json")
category_CA_df = pd.read_json(json_CA)
CA_df = json_normalize(category_CA_df['items'])

In [3]:
json_US = os.path.join("data", "US_category_id.json")
category_US_df = pd.read_json(json_US)
US_df = json_normalize(category_US_df['items'])

In [5]:
json_MX = os.path.join("data", "GB_category_id.json")
category_GB_df = pd.read_json(json_MX)
GB_df = json_normalize(category_GB_df['items'])

In [6]:
csv_GB = os.path.join("data", "GBvideos.csv")
GB_csv_df = pd.read_csv(csv_GB)

In [7]:
csv_US = os.path.join("data", "USvideos.csv")
US_csv_df = pd.read_csv(csv_US)

In [8]:
csv_CA = os.path.join("data", "CAvideos.csv")
CA_csv_df = pd.read_csv(csv_CA)

In [9]:
US_df.head()

,etag,id,kind,snippet.assignable,snippet.channelId,snippet.title
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation
1,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles
2,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Music
3,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals
4,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Sports


In [10]:
US_csv_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
